In [ ]:
!pip install -r requirements.txt

In [ ]:
import ray
ray.init(runtime_env={"pip": ["langchain", "pypdf", "sentence_transformers", "transformers"]})

In [ ]:
from langchain.document_loaders import ArxivLoader

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000, chunk_overlap=100, length_function=len
    )

In [ ]:
!pip install certifi

In [ ]:
docs_chunks = ArxivLoader(query="1605.08386", load_max_docs=2).load_and_split(text_splitter)

In [ ]:
ds = ray.data.from_items(docs_chunks)

In [ ]:
for d in ds.iter_rows():
  print(d)

In [ ]:
# preprocess chunks by replacing \n by empty space
ds_batch = ds.flat_map(lambda row: [{'item': row["item"].page_content.replace("\n", " ")}])

In [ ]:
for row in ds_batch.iter_rows():
  print(row)

print("dec19")

In [ ]:
from sentence_transformers import SentenceTransformer
from typing import List

In [ ]:
model_name="intfloat/multilingual-e5-small"

In [ ]:
class Embed:
  def __init__(self):
        self.transformer = SentenceTransformer(model_name)

  def __call__(self, text_batch: List[str]):
      # We manually encode using sentence_transformer since LangChain
      # HuggingfaceEmbeddings does not support specifying a batch size yet.
      text = text_batch["item"]
      embeddings = self.transformer.encode(
          text,
          batch_size=100 #,  # Large batch size to maximize GPU/CPU utilization.
          #device="cuda",
      ).tolist()

      #return {'results': list((text, embeddings))}
      return {'results':list(zip(text, embeddings))}


In [ ]:
ds_embed = ds_batch.map_batches(
    Embed,
    compute=ray.data.ActorPoolStrategy(size=1),
    # Large batch size to maximize GPU utilization.
    # Too large a batch size may result in GPU running out of memory.
    # If the chunk size is increased, then decrease batch size.
    # If the chunk size is decreased, then increase batch size.
    #batch_size=10,  # Large batch size to maximize GPU utilization.
    #compute=ray.data.ActorPoolStrategy(min_size=1, max_size=20),  # I have 20 GPUs in my cluster
    #num_gpus=0.1,  # 1 GPU for each actor.
    num_cpus=0,
)

In [ ]:
for output in ds_embed.iter_rows():
    print(len(output["results"][1]))

## Setup connection with Astra DB

In [ ]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from local_creds_secrets import *

cloud_config= {
  'secure_connect_bundle': secure_bundle_path
}
auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

row = session.execute("select release_version from system.local").one()
if row:
  print(row[0])
else:
  print("An error occurred.")

In [ ]:
table_name = f"{db_keyspace}.papers"

In [ ]:
table_creation_query = f"""CREATE TABLE IF NOT EXISTS {table_name} (
  id int PRIMARY KEY,
  name TEXT,
  description TEXT,
  item_vector VECTOR<FLOAT, 384> //create a 384-dimensional embedding
);"""

In [ ]:
session.execute(table_creation_query)

In [ ]:
sai_index_query= f"""CREATE CUSTOM INDEX IF NOT EXISTS ann_index
  ON {table_name}(item_vector) USING 'StorageAttachedIndex'; """
  #WITH OPTIONS = { 'similarity_function': 'DOT_PRODUCT' };"""
session.execute(sai_index_query)

In [ ]:
session.execute(sai_index_query)

### Insert vector records into AstraDB

In [ ]:
id=1
for output in ds_embed.iter_rows():
  query = f"""INSERT INTO {db_keyspace}.papers (id, name, description, item_vector)
  VALUES ({id},'paper_1_sample', '{output["results"][0]}',{output["results"][1]});"""
  id+=1
  session.execute(query)

In [ ]:
rows = session.execute(f"select * from {db_keyspace}.papers limit 3;").all()

In [ ]:
for row in rows:
  print(row)

## Searching a sentence from the table

In [ ]:
model = SentenceTransformer('intfloat/multilingual-e5-small')

In [ ]:
example_sentence="two lattice points are connected by an edge if their diﬀerence lies in a ﬁnite"
embedding = list(model.encode(example_sentence))


In [ ]:
search_query = f"""SELECT * FROM {db_keyspace}.papers
ORDER BY item_vector ANN OF {embedding}
LIMIT 2;"""

In [ ]:
session.execute(search_query).all()